In [1]:
"""
requirements:
 pip install pandas janome nlpplot matplotlib plotly
"""
import pandas as pd
import nlplot
import pandas as pd
import plotly
from plotly.subplots import make_subplots
from plotly.offline import iplot
import matplotlib.pyplot as plt

import word_wakati as wkt

In [2]:
df = pd.read_csv('Dataset.csv')
df.head()

,年度,学科／研究科,研究室,学籍番号,賞,氏名,要旨,題目
0,H17,情報科学科,相研,02JK001,NaN,青木雄司,NaN,WEBページにおけるユーザビリティの追及
1,H17,情報科学科,松永研,02JK002,NaN,青柳茂樹,NaN,安全運転管理教育システム(ASSIST)における複数カメラ使用での交通事故防止対策
2,H17,情報科学科,宮崎研,02JK003,NaN,芦刈綾子,NaN,電子透かしの体制に関する検討 -加重平均フィルタ・メディアンフィルタの場合-
3,H17,情報科学科,成研,02JK005,NaN,足達祐治,NaN,対面教育を支援するためのウェブ助言システム
4,H17,情報科学科,石田健研,02JK006,NaN,安部弘治,NaN,衛星通信検討のための気象データの解析-2002年のデータ解析-


In [29]:
years = df["年度"].unique()
y, d = 5, 2 # loop for every 'y' years
for i in range(0,len(years)-y, d):
    print(years[i],years[i+y],)

H17 H22
H19 H24
H21 H26
H23 H28
H25 H30
H27 R02
H29 R04


In [4]:
len(df)

2148

In [5]:
fake_lab={'成研': 'A研', '石田健研': 'B研', '仲研': 'C研', '澤田研': 'D研', 
          '稲永研': 'E研', '合志研': 'F研', '米元研': 'G研', '朝廣研': 'H研', 
          '下川研': 'M研', '安部研': 'N研', 'アプドゥハン研': 'P研', '田中研': 'Q研', '安武研': 'R研'}
print(fake_lab.values())

dict_values(['A研', 'B研', 'C研', 'D研', 'E研', 'F研', 'G研', 'H研', 'M研', 'N研', 'P研', 'Q研', 'R研'])


# various graphs
- themeriver graph (mathplotlib.stackplots)
- knowledge structure graph

In [6]:
import importlib
importlib.reload(wkt)
mecab = wkt.create_parser(worker='mecab', parts_of_speech=['名詞','動詞','形容詞'])
janome = wkt.create_parser(worker='janome', parts_of_speech=['名詞','動詞','形容詞'])
text = '拡張現実感のための実物体の奥行きを考慮した陰面消去'
rs_j = wkt.word_seq(text,parser=janome)
rs_m = wkt.word_seq(text,parser=mecab)
print(rs_j)
print(rs_m)

['拡張', '現実', '感', 'ため', '物体', '奥行き', '考慮', 'する', '陰', '面', '消去']
['拡張現実', '感', 'ため', '物体', '奥行き', '考慮', 'した陰', '面', '消去']


In [7]:
text = '研究室配属における学生の研究室に対する理解を深める情報共有システム'
rs_j = wkt.word_seq(text,parser=janome)
rs_m = wkt.word_seq(text,parser=mecab)
print(rs_j)
print(rs_m)

['研究', '室', '配属', '学生', '研究', '室', '理解', '深める', '情報', '共有', 'システム']
['研究室', '配属', '学生', '研究室', '理解', '深める', '情報共有', 'システム']


In [71]:
%matplotlib inline
import importlib
importlib.reload(wkt)

targets = ['動詞', '名詞', '形容詞'] #評価対象にする品詞
ignores = ['－', '～', '-', '(', ')', '---', '.','する','ため','用いる','有する','用','使う'] #無視する記号や単語
mecab  = wkt.create_parser(worker='mecab',  parts_of_speech=targets, stop_words=ignores)

In [147]:
import collections

def ngram(words, n):
    lists = []
    m = len(words)
    if m > n:
        for i in range(n):
            lists += [ words[i:m-i+n] ]
        glist = list(zip(*lists))
    else:
        glist = words
        
    return [" ".join(g) for g in glist]

ngram(['a','b', 'c', 'g'], 3)

['a b c', 'b c g']

In [163]:
def topk(i, j, n, stopwords=0):
    y1, y2 = years[i],years[j]
    df1 = None
    df1 = df[(df['研究室'].str.contains("研")) & (df['年度']>=y1) & (df['年度']<=y2)]
    titles =  [ t[0] for t in df1.loc[:,['題目']].values]
    freq = collections.defaultdict(int)
    for text in titles:
        words = wkt.word_seq(text, parser=mecab)
        glist = ngram(words, 3)
#         print(glist)
        for g in glist:
            freq[g] += 1 
    freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)   
    return freq[stopwords:n]

k, s = 40, 1
top10 = topk(0,7,k,stopwords=s)
print(top10)
print()
top10 = topk(10,17,k,stopwords=s)
print(top10)

[('管理教育 システム ASSIST', 29), ('支援 システム 開発', 17), ('電磁波 伝搬 シミュレータ', 12), ('安全運転 助言 検査', 11), ('ドライビング シミュレータ 開発', 10), ('式 安全運転 助言', 10), ('問題 メタヒューリスティクス 実験的', 9), ('メタヒューリスティクス 実験的 解析', 9), ('教育 ドライビング シミュレータ', 9), ('作成 支援 システム', 8), ('組合せ最適化 問題 メタヒューリスティクス', 8), ('FDTD 法 利用', 8), ('伝搬 シミュレータ 作成', 8), ('KM 式 安全運転', 8), ('体験型 ドライビング シミュレータ', 8), ('CO2 排出 量', 8), ('排出 量 削減', 8), ('講義 記録 システム', 7), ('移動 効率 体験型', 7), ('効率 体験型 ドライビング', 7), ('Java 数学教育 支援', 7), ('交通 標識 抽出', 6), ('クラスタ コンピューティング 教育支援', 6), ('運営 支援 システム', 6), ('足型 測定 器', 6), ('法 利用 2次元', 6), ('利用 2次元 電磁波', 6), ('2次元 電磁波 伝搬', 6), ('型 KM 式', 6), ('道路交通 標識 認識', 6), ('数学教育 支援 ソフトウェア', 6), ('衛星 通信 検討', 5), ('記録 システム アクセス解析', 5), ('システム 設計 開発', 5), ('マルチ ビーム 型', 5), ('ビーム 型 衛星', 5), ('型 衛星 通信', 5), ('一時停止 教育 ドライビング', 5), ('動的 挙動 解析', 5)]

[('支援 システム 開発', 27), ('安全運転 管理教育 システム', 21), ('管理教育 システム ASSIST', 21), ('地域 公共 交通', 17), ('教育 ドライビング シミュレータ', 16), ('公共 交通 基盤', 16), ('交通 基盤 データ', 16), ('Web システム 開発', 15), ('アルゴリズム 実装 実験', 14), ('計